- pretty view git messages ala nbdime
- simplified notebook workflow

- nb2repo: save current notebook to repo dir
- drive_nb2repo_nb: all previously committed drive notebooks to repo
- git_commit, etc. do nb2repo first
- set up global git email and user
- github CLI download and a few functions, create private repo, create issue...


In [1]:
import os

# get environment

In [2]:
# %%snippet -s -r -e first_cell
if 'c' not in globals():
  !pip install nbf
  from nbf import epc
  c = epc.do_setup({'resources':{'nbf':{}}, 'keyring':'bit.ly/3QPP1bs'})
  %run {c.get('nb_dir')}notebooks_as_features.ipynb
  logger = epc.get_logger(c.get('nb_name'))
else: 
  !cd {c['project_path']} && git pull

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2023-01-21 18:01:44 epc INFO     epc.config_dict(c): may be overridden by `c[setting_name]`
2023-01-21 09:01:44 epc INFO     log_level: DEBUG
2023-01-21 09:01:44 epc INFO     timezone: America/Anchorage
2023-01-21 09:01:44 epc INFO     use_google_drive: False
2023-01-21 09:01:44 epc INFO     epc: U2FsdGVkX1+q8onitIzydG8ZC+SdDrKgTQQl1L91LHTkNXknnga+XIEkZk8KRPG4gMpief6NNazpIu2RK3IAYKeo3B4UCQsbQLm78nktkSxt/ZIYpnneoDvD/J3vYoCDLDExrEJ8AIVAdS6n9HBKHl/4ouEA78XQc+vh5qb1nkvneiBX3LYK01KQh0SMwJQqd/o3IU9G2o6cyEur0Ys6GT6P5qX8Osg11wlJkPsPPT6wlb2OI/HZA+oouuT2/RIoHJr0BdgxOZ+3dbjOaXj5GVC2NImuqRZFQuR0CyV5tQriNioxt6Lp3gqoRxsfWLCWNOpH2EgErEdeu8qmWyrp3iRa0MaxQ8MBmDO86ozE8a+3BlNit1yx1W0FHJVOBsTP8L6qD14rMmuqHuUgvCtLs2KGss6wu6hrdiqFeohqQ2KdE39ahW+xpBW5osO3d5ZXbGCQjmH3czneZ8vnGZDXD8JKEOcZIy31FmVGNkp0LwghGDmhjNmYLwnxTy5Zgbjh/P8+PxHaKLJzwv+r7a4YCunVOb2503kZiWuikb2XpKSTPTIgX38g7l28E+sgIzrzMZivf7+gTxJv3LViG/K0eL6x+NAuYosT9xRfDPLXIJDeIEFTD7/1uWqO5heicQo/NoOHl8WcxMqk1RSl/kzpe3ldyb2CCSroGQSbrJAAOb7gJS2/rmyrwE4UFo3ojf

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 13 (delta 7), reused 12 (delta 6), pack-reused 0
Unpacking objects: 100% (13/13), done.
From https://github.com/Jaciss/nbf
   33733ab..74d62a4  main       -> origin/main
Updating 33733ab..74d62a4
Fast-forward
 notebooks/notebooks_as_features.ipynb | 554 +++++++++++++++++++++++++++++++++-
 pyproject.toml                        |  18 ++
 tests/test_d2d_titlenames.py          |  22 ++
 3 files changed, 593 insertions(+), 1 deletion(-)
 create mode 100644 pyproject.toml
 create mode 100644 tests/test_d2d_titlenames.py


2023-01-21 09:01:46 git_for_colab INFO     root notebook: git_for_colab


In [3]:
# c

# Initial Config

In [4]:
git_user = epc.get_token('github_user')
git_email = epc.get_token('github_email')
!git config --global user.email "{git_email}"
!git config --global user.name "{git_user}"

Save passphrase as variable, leave blank to enter each time: ··········


## nbdime for notebook git diff
`git diff` and similar on JSON that has many minute changes is nearly unworkable. This eliminates much of the noise.

In [5]:
%%capture nbdime_install_output
nv = !which nbdime
# print('which nbdime:',str(nv))
if str(nv).strip()=='[]':
  %pip install nbdime
  !nbdime config-git --enable --global

In [6]:
%%writefile /root/.jupyter/nbdime_config.json
{
  "GitDiff": {
    "Ignore": {
      "/cells/*/outputs": true,
      "/cells/*":["execution_count"],
      "/cells/*/metadata": ["collapsed", "autoscroll", "deletable", "editable", "executionInfo", "outputId", "colab"]
    }
  }
}

Overwriting /root/.jupyter/nbdime_config.json


# Generic Git Functions

In [19]:
import json
from google.colab import _message

def get_repo_name(): #idx:int=0):
  return c.get('active_project')
  # try:
  #   repo = c['github'][idx]
  #   return repo
  # except IndexError:
  #   logger.error("No repo name at c['github']["+str(idx)+"]")
  #   return None

# get_repo_name()

def git_pull(quiet=False):
  repo = c.get('active_project')
  if repo is not None:
    if quiet is False:
      !cd {c['projects_dir']}{repo} && git status
    !cd {c['projects_dir']}{repo} && git pull

# git_pull()

def nb2repo(show_diff=True):
  '''this uses projects_dir/active_project/notebooks vs nb_dir to avoid writing
  colab notebooks, which are automatically saved; copy colab notebooks to the repo dir'''
  repo = c.get('active_project')
  
  if repo is not None:
    repo_dest_dir = c.get('nb_dir').replace('\ ',' ') #repo+'/'+ 
    if c['use_google_drive']==True: repo_dest_dir = c['projects_dir']+repo+'/notebooks/'
    hate = f"{repo_dest_dir}{c['running_nb']}.ipynb"
    print('try to write')
    print(r""+hate)
    activenb = _message.blocking_request('get_ipynb', request='', timeout_sec=5)['ipynb']
    with open(hate, 'w') as nb_file:
      nb_file.write(json.dumps(activenb,indent=2))
    hate = hate.replace(' ','\ ')
    !ls -lart {hate} |tail -1
    if show_diff == True:
      !cd {c['projects_dir']}{repo} && echo "" && git diff --stat && echo -e "\033[0m\n" && git diff --minimal

# nb2repo()

def git_commit(message:str='from github colab notebook'):
  '''add -A, status w/confirmation, commit'''
  repo = get_repo_name()
  if repo is not None:
    nb2repo() # save current nb
    git_pull(True) # pull any changes
    !cd {c['projects_dir']}{repo} && git add -A # add or changes
    !cd {c['projects_dir']}{repo} && git status # take a look at what we have
    print('message:',message)
    yn = input('Do you want to commit now? y/N: ')
    if yn=='y':
      !cd {c['projects_dir']}{repo} && git commit -m "{message}"

# git_commit()

def git_ppush(message:str='from github colab notebook'):
  '''add, commit, push all in one'''
  repo = c.get('active_project')
  if repo is not None:
    git_commit(message)
    !cd {c['projects_dir']}{repo} && git push

# git_ppush('typehinting')

def git_nbastatus():
  repo = c.get('active_project')
  if repo is not None:
    nb2repo(False)
    !cd {c['projects_dir']}{repo} && git add -A && echo "diff from origin:" && git diff origin --stat && echo -e "\033[0m\n" && git status

def drive_nb2repo_nb(message='update all from colab notebooks/*'):
  '''for file in projects_dir/active_project/notebooks/*.ipynb copy Google Drive 
  "Colab Notebooks/file.ipynb" on top of it'''
  from google.colab import drive
  os.path.isdir('/content/drive/') or drive.mount('/content/drive/')
  enbooks = !ls {c['active_project']}/notebooks/*.ipynb
  for book in enbooks:
    nbfname = os.path.basename(book)
    # cmd = f"cp drive/MyDrive/Projects/{c['active_project']}/notebooks/{nbfname} {book}"
    cmd = f"cp drive/MyDrive/Colab\ Notebooks/{nbfname} {book}"
    print(cmd)
    !{cmd}
  git_ppush(message)
  # gnbooks = !ls drive/MyDrive/Projects/{c['active_project']}/notebooks/*.ipynb
  # print(gnbooks)
  # drive/Colab\ Notebooks/
# drive_nb2repo_nb()

In [8]:
# !cd {c.get('project_path')} && git pull
# nb2repo()
# git_ppush('remove defunct notebooks')

# GitHub CLI

In [9]:
def get_gh():
  '''installs GitHub CLI'''
  hazgh = !which gh
  if len(hazgh)==0:
    command = '''
    sudo apt install curl -y
    curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg \
    && sudo chmod go+r /usr/share/keyrings/githubcli-archive-keyring.gpg \
    && echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null \
    && sudo apt update \
    && sudo apt install gh -y
    '''
    !{command} > /dev/null
    ghtoken = epc.get_token('github')
    !echo "{ghtoken}" | gh auth login --with-token

In [10]:
def create_private_github_repo(name):
  '''creates a private repo, relies on epc'''
  get_gh()
  user = epc.get_token('github_user')
  !gh repo create {name} --private
  # private! --clone in above does not work =P
  epc.github_fetch({'repo':name,'user':git_user,'token':epc.get_token('github'),'filepath':''})
  !cd {name} && echo "# {name}" >> README.md 
  !cd {name} && git add README.md && git commit -m "initial commit"
  !cd {name} && git branch -M main
  !cd {name} && git push -u origin main
  !cat {name}/.git/config
  !cd {name} && git status
# !cd {c['nb_dir']} && git branch --set-upstream-to=origin/dev dev

# create_private_github_repo('gotit')

In [11]:
import os
def create_issue(title,body,label='bug',a='@me',idx=0):
  '''common labels: bug, enhancement, documentation, question'''
  get_gh()
  repo = get_repo_name(idx)
  response = !cd {repo} && gh issue create -t "{title}" -b "{body}" -a "{a}" -l "{label}"
  print(' '.join(response))
  if len(response)==4: return os.path.basename(response[3])
  return response

# create_issue('fix knowledge of GitHub closes syntax','this is awesome','enhancement')

# Misc

In [12]:
# make a copy of active, nbdiff it with what's in the repo dir:
# nb2repo(0,'../../tmp/',False)
# !cd jai && nbdiff notebooks/git_for_colab.ipynb ../../tmp/git_for_colab.ipynb

# to try:
# !cd jai && nbdime mergetool

# failed to get nbdiff-webweb diff working: https://colab.research.google.com/drive/1frMmd0WSaXhOQXef0PtTWojCXuuvMxur

# diff repo with remote
# !cd jai && git diff main:notebooks/git_for_colab.ipynb notebooks/git_for_colab.ipynb

In [13]:
# if you want everything in the root directory:
# !mv {repo}/* .
# !mv {repo}/.[!.]* .
# !rm -rf {repo}

## diff live vs github files

In [14]:
# DIFF Live vs Git
# # git version
# text_file = open("jai/notebooks/encrypted_private_config.ipynb", "r")
# data = text_file.read()
# text_file.close()
# jsonFile = open("gitnb.json", "w")
# jsonFile.write(json.dumps(json.loads(data),indent=2))
# jsonFile.close()

# # live version
# activenb = _message.blocking_request('get_ipynb', request='', timeout_sec=5)['ipynb']
# jsonFile = open("activenb.json", "w")
# jsonFile.write(json.dumps(activenb,indent=2))
# jsonFile.close()

# # !head gitnb.json
# !diff -c activenb.json gitnb.json
# !cp activenb.json jai/notebooks/encrypted_private_configTEST.ipynb


# EOF

In [15]:
# %%writerun {nb_dir}templates/git_message -e
#@title Commit Message { run: "auto", vertical-output: true }
commit_message = "wip" #@param {type:"string"}
commit_message = c.get('commit_message',commit_message+' |colab')
commit_message

'wip |colab'

In [ ]:
# %%writerun {c['nb_dir']}templates/git -e
#@title Git Actions { run: "auto", vertical-output: true }
if c['running_nb'] == c['nb_name']:
  github_index = 0 #@param ["0", "1", "2"] {type:"raw"}
  git_action = "nb2repo*" #@param ["","nb2repo*","pull","diff","diff origin --minimal","git_nbastatus*","commit -m {message}","push","ppush*","status","fetch","merge","push origin dev","diff main dev","merge main","merge dev","checkout main","checkout dev","diff --staged","restore","add .","branch -a","log -15 --graph --oneline --decorate","reset --hard HEAD","reset --hard HEAD~1","commit --amend -m {messaage}"]

  # print('repo indices: '+str(dict(zip(range(len(c['github'])),c['github']))))
  # if github_index > len(c['github'])-1:
  #   print('no repo at that index')
  # else:
  # print('repo:',c['github'][github_index])
  if git_action!='':
    if git_action=='nb2repo*': nb2repo()
    elif git_action=='ppush*': git_ppush(commit_message)
    elif git_action=='git_nbastatus*': git_nbastatus()
    else:
      if git_action=="commit -m {message}": git_action = f"commit -m '{commit_message}'"
      if git_action=="commit --amend -m {messaage}": git_action = f"commit --amend -m '{commit_message}'"
      !cd {c['project_path']} && git {git_action}

try to write
/content/nbf/notebooks/git_for_colab.ipynb


In [17]:
# for automatic pushes
# if c['running_nb'] == c['nb_name']:
#   git_ppush('colab')

In [18]:
# c['nb_name'] = c['nb_run_by'] # give nb_name back to any calling notebooks